# Geoembedding-based Land Cover Modelling Pipeline

**Author:** Pablo Timoner  
**Date:** 2026-01-20

This notebook implements a **land cover classification workflow** using reference data (ESA World Cover or CSV tables) and raster-based geoembeddings. The main steps are:

## 1. Reference Data Preparation
- Load reference data: either a **CSV table** or an **ESA World Cover raster** (`is_esa=True`).
- If ESA raster:
  - Mask `nodata` pixels.
  - Compute class distribution and proportionally allocate sample points per class.
  - Perform stratified random sampling of pixel locations.
  - Convert sampled pixels to geographic coordinates (`lon/lat`).
  - Create a DataFrame for further processing
- If the embedding raster CRS differs from WGS84:
  - Transform coordinates to match embedding CRS.
  - Add `x_<EPSG>` / `y_<EPSG>` columns for modelling.
- Reclassify land cover classes using a mapping table.

## 2. Sampling & Feature Extraction
- Keep only points that fall within the embedding raster extent.
- **Extract embedding values** at sampled point locations.

## 3. Training / Test Split
- Limit total number of samples (`n_sample`) to reduce computational cost.
- Remove classes with extremely low representation (<1% of samples).
- Split sampled points into **training and testing sets**.
- Stratify split by land cover class to preserve class proportions.

## 4. Random Forest Model
- Initialize Random Forest classifier.
- Stage 1 hyperparameter tuning(tree-level):
  - `max_features`, `max_depth`, `min_samples_leaf`
- Stage 2 hyperparameter tuning (forest-level):
  - `n_estimators`, `min_samples_split`, `class_weight`
- Select best hyperparameters and fit **final model** on all samples.

## 5. Raster Prediction
- Predict land cover for large raster in **blocks** to save memory (`block_shape` can be adapted).
- Steps per block:
  1. Read block from embedding raster.
  2. Flatten block to `(pixels x features)` format.
  3. Apply trained Random Forest to predict class labels.
  4. Reshape predictions to block shape and write to output raster.
- Continue iteratively until the entire raster is processed.
- Save final predicted raster as `.tif`.

## Notes
- Designed to handle **large rasters** and **multiple embedding bands** efficiently.
- Fully reproducible with **stratified sampling**, **fixed random seed**, and saved hyperparameters.
- Can handle any **EPSG CRS** for embeddings and reference data.

In [ ]:
import os
import rasterio
import pandas as pd
import geopandas as gpd
import numpy as np
import joblib
from rasterio.transform import xy
from pyproj import Transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [ ]:
# Load reference data (can be a CSV table or the path to a ESA World Cover raster)
# If ESA World Cover must be EPSG:4326
ref_path = "/home/jupyterhome/jupyter-pablotimoner/Geoembeddings/Manaus/ESA_WorldCover_2021_manaus.tif"

# Is the reference data a raster of the ESA World Cover
is_esa = True

# If is_esa = True, how many pixels to potentially be considered for modelling (calibration & validation)
n_esa_sample = 10000

# If is_esa = False names of columns in the reference data table where we have the coordinates, and where we have the land cover class
X_col = 'lon'
Y_col = 'lat'
LC_col = 'AS_17'

# Geoembedding path
embedding_path = "/home/jupyterhome/jupyter-pablotimoner/Geoembeddings/Manaus/EE_exports/embedding_2020_merged.tif"

# Land cover class mapping table (requires at least two columns: 'code' for original code and 'value' for new code)
class_table_path = "/home/jupyterhome/jupyter-pablotimoner/Geoembeddings/LC_classes/CLASSES_ESA.csv"

# Total number of observations used for modelling (limited to balance computational cost)
# When is_esa=True and n_esa_sample is smaller than n_sample,
# n_sample will be reduced automatically to match n_esa_sample
n_sample = 5000

# Number of cores for parallel computing (RAM is usually the limiting factor ; keep the number of cores low)
ncore = 2

# Output folder path
output_folder = "/home/jupyterhome/jupyter-pablotimoner/Geoembeddings/Manaus/results/AlphaEarth/test"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Names for intermediate output files (without extension)
sample_filename = 'samples'
final_df_filename = 'df_final'
best_params_i_filename = 'best_params1'
best_params_ii_filename = 'best_params2'
report_filename = 'report'
best_model_filename = 'best_model'
prediction_filename = 'lc_prediction'

In [ ]:
# Read or prepare the reference data
if not is_esa:
    df = pd.read_csv(ref_path)
else:
    print("Preparing the reference data using ESA World Cover...")
    with rasterio.open(ref_path) as src:
        data = src.read(1)
        transform = src.transform
        crs = src.crs
        nodata = src.nodata
    
    # Mask nodata
    if nodata is not None:
        mask = data != nodata
        data_valid = data[mask]
    else:
        data_valid = data
    
    # # Compute class distribution from valid pixels
    classes, counts = np.unique(data_valid, return_counts=True)
    total_pixels = counts.sum()
    proportions = counts / total_pixels
    
    # Allocate number of samples per class proportionally
    samples_per_class = np.round(proportions * n_esa_sample).astype(int)
    
    # Adjust to reach exactly n_esa_sample (e.g. 20,000)
    diff = n_esa_sample - samples_per_class.sum()
    samples_per_class[np.argmax(samples_per_class)] += diff
    
    # Get row/column indices of valid pixels
    rows, cols = np.where(mask if nodata is not None else np.ones_like(data, dtype=bool))

    # Stratified random sampling of pixel locations per class
    df_list = []
    
    for cls, n_cls in zip(classes, samples_per_class):
        if n_cls == 0:
            continue
        # Indices of pixels belonging to the current class
        idx = np.where(data[rows, cols] == cls)[0]
        # Ensure sampling without replacement
        if len(idx) < n_cls:
            n_cls = len(idx)
        
        chosen = np.random.choice(idx, size=n_cls, replace=False)
        
        # Convert pixel indices to geographic coordinates
        r = rows[chosen]
        c = cols[chosen]
        lon, lat = xy(transform, r, c)

        # Store sampled points
        df_list.append(
            pd.DataFrame({
                "esa": cls,
                "lon": lon,
                "lat": lat
            })
        )
    # Create reference dataframe
    df = pd.concat(df_list, ignore_index=True)
    LC_col = "esa"
    X_col = 'lon'
    Y_col = 'lat'
    with rasterio.open(embedding_path) as src:
        crs = src.crs
        if crs.is_epsg_code:
            embedding_epsg = crs.to_epsg()
            if embedding_epsg != 4326:
                # Create transformer from WGS84 to embedding CRS
                transformer = Transformer.from_crs(4326, embedding_epsg, always_xy=True)
                # Apply the transformation
                df[f'x_{embedding_epsg}'], df[f'y_{embedding_epsg}'] = transformer.transform(
                    df['lon'].values, df['lat'].values
                )
                X_col = f'x_{embedding_epsg}'
                Y_col = f'y_{embedding_epsg}'
            else:
                pass     
        else:
            raise ValueError("CRS does not have an EPSG code. Cannot determine embedding EPSG.")

In [ ]:
# Reclassifcation of Land cover class
classes_df = pd.read_csv(class_table_path)
code_to_value = dict(zip(classes_df["code"], classes_df["value"]))
df["value"] = df[LC_col].map(code_to_value)

In [ ]:
# Only keep points that fall within the extent of the embeddings
with rasterio.open(embedding_path) as src:
    print("CRS:", src.crs)
    print("Bounds:", src.bounds)
    print("Band count:", src.count)
    
    xmin = src.bounds.left
    xmax = src.bounds.right
    ymin = src.bounds.bottom
    ymax = src.bounds.top

mask = (
    (df[X_col] >= xmin) &
    (df[X_col] <= xmax) &
    (df[Y_col] >= ymin) &
    (df[Y_col] <= ymax)
)

df_in = df[mask].copy()

print("Points before filtering:", len(df))
print("Points after filtering:", len(df_in))
print(f"Kept: {len(df_in)/len(df):.2%}")

In [ ]:
# Sample embedding values at point locations
with rasterio.open(embedding_path) as src:
    coords = list(zip(df_in[X_col], df_in[Y_col]))
    samples = np.array(list(src.sample(coords)))
    
print("Samples shape:", samples.shape)

In [ ]:
# Optional 
np.save(os.path.join(output_folder, f"{sample_filename}.npy"), samples)

In [ ]:
# Optional (in case values already sampled)
samples = np.load(os.path.join(output_folder, f"{sample_filename}.npy"))

In [ ]:
# Create data frame with sampled values
band_cols = [f"band_{i+1}" for i in range(src.count)]
embeddings_df = pd.DataFrame(samples, columns=band_cols)

In [ ]:
# Create final dataframe for modelling
# Safety step: remove rows where all embedding values are 0
embeddings_df = embeddings_df[~(embeddings_df == 0).all(axis=1)].reset_index(drop=True)

# Concatenate 'value', LC_col, and embeddings into a single dataframe
df_final = pd.concat(
    [df_in[['value', LC_col]].reset_index(drop=True),
     embeddings_df.reset_index(drop=True)],
    axis=1
)

# Remove rows with NaN values in 'value'
df_final = df_final[df_final['value'].notna()].reset_index(drop=True)
print(df_final.head())
df_final.shape

In [ ]:
# Optional: Save cleaned DataFrame
output_file = os.path.join(output_folder, f"{final_df_filename}.csv")
df_final.to_csv(output_file, index=False)

In [ ]:
# Optional (in case we already have a saved final DataFrame)
output_file = os.path.join(output_folder, f"{final_df_filename}.csv")
df_final = pd.read_csv(output_file)

In [ ]:
# If using ESA data and requested n_sample is larger than available ESA samples,
# automatically reduce n_sample to n_esa_sample
if is_esa and n_sample > n_esa_sample:
    n_sample = n_esa_sample

# Class proportions (based on non-aggregated LC classes)
class_props = df_final[LC_col].value_counts(normalize=True)

# Stratified sampling
samples = []

for cls, prop in class_props.items():
    # Determine number of samples for this class (at least 1)
    n_cls = max(1, int(round(prop * n_sample)))
    # Randomly sample n_cls rows for the current class
    samples.append(
        df_final[df_final[LC_col] == cls].sample(n=n_cls, random_state=42)
    )
    
# Concatenate all class-specific samples, shuffle (to prevent potential order bias), and limit total to n_sample
df_sample = (
    pd.concat(samples)
      .sample(n=min(n_sample, sum(len(s) for s in samples)), random_state=42)
      .reset_index(drop=True)
)

df_sample.shape

In [ ]:
# Features for modelling (drop target and original class columns)
X = df_sample.drop(columns=['value', LC_col])
# Target
y = df_sample['value']
# Proportion of each class
y.value_counts(normalize=True)

In [ ]:
# Remove classes with less than 1%
to_remove = y.value_counts()[y.value_counts(normalize=True) < 0.01].index
for cls in to_remove:
    indices = y[y == cls].index
    X = X.drop(index=indices)
    y = y.drop(index=indices)
# Proportion of each class   
y.value_counts(normalize=True)

In [ ]:
# Split sampled data into training and testing sets
# 20% test size, stratified by the target class to preserve class proportions
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
# Number of predictors (used further for model tuning)
n_features = X_train.shape[1]
print("Number of features in the embedding:", n_features)

In [ ]:
# Initialize Random Forest classifier
# Out-of-bag score disabled (we use cross-validation for hyperparameter tuning)
rf_base = RandomForestClassifier(
    random_state=42,
    n_jobs=ncore,
    oob_score=False,
    bootstrap=True
)

In [ ]:
# Model tuning
# Stage 1: Tune individual tree parameters
# Define the parameter grid for max_features, max_depth, and min_samples_leaf
# This affects tree structure and feature selection per split
param_grid_stage1 = {
    'max_features': ['sqrt', 'log2', max(1, n_features // 16), max(1, n_features // 8), max(1, n_features // 4)],
    'max_depth': [None, 20, 40],
    'min_samples_leaf': [1, 5, 10]
}

# Set up grid search with 5-fold cross-validation and macro F1 score for multiclass
grid_stage1 = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid_stage1,
    cv=5,
    scoring='f1_macro',
    n_jobs=ncore,
    verbose=1
)

# Run the grid search
grid_stage1.fit(X_train, y_train)

# Save Stage-1 best parameters to a text file
best_stage1_params = grid_stage1.best_params_
params_path = os.path.join(output_folder, f"{best_params_i_filename}.txt")
with open(params_path, "w") as f:
    f.write("Best parameters\n")
    f.write("===================\n\n")
    for key, value in best_stage1_params.items():
        f.write(f"{key}: {value}\n")

# Initialize a new Random Forest with Stage-1 best params for Stage 2
rf_stage2_base = RandomForestClassifier(
    **best_stage1_params,
    random_state=42,
    n_jobs=ncore,
    bootstrap=True
)

# Stage 2: Tune forest-level parameters
# Parameter grid for n_estimators, min_samples_split, and class_weight
# This stage focuses on stability and handling class imbalance
param_grid_stage2 = {
    'n_estimators': [400, 800, 1200],
    'min_samples_split': [2, 10],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

# Set up Stage-2 grid search with 5-fold CV
grid_stage2 = GridSearchCV(
    estimator=rf_stage2_base,
    param_grid=param_grid_stage2,
    cv=5,
    scoring='f1_macro',
    n_jobs=ncore,
    verbose=1
)

# Run the grid search
grid_stage2.fit(X_train, y_train)

# Save Stage-2 best parameters to text file
best_params = grid_stage2.best_params_
params_path = os.path.join(output_folder, f"{best_params_ii_filename}.txt")
with open(params_path, "w") as f:
    f.write("Best Hyperparameters\n")
    f.write("===================\n\n")
    for key, value in best_params.items():
        f.write(f"{key}: {value}\n")

# Retrieve the best Random Forest from Stage 2
best_rf = grid_stage2.best_estimator_

# Predict on the test set
y_pred = best_rf.predict(X_test)

# Classification report
report = classification_report(y_test, y_pred)
# Save report to text file
report_path = os.path.join(output_folder, f"{report_filename}.txt")
with open(report_path, "w") as f:
    f.write("Classification Report\n")
    f.write("=====================\n\n")
    f.write(report)

# Get all the parameters
best_params = grid_stage2.best_estimator_.get_params()

# Initialize the classifier
best_rf_final = RandomForestClassifier(
    **best_params
)

# Fit the final model
best_rf_final.fit(X, y)

In [ ]:
# Optional: Save final model
model_path = os.path.join(output_folder, f"{best_model_filename}.pkl")
joblib.dump(best_rf_final, model_path)

In [ ]:
# Optional (in case we have already calibrated and saved the model)
model_path = os.path.join(output_folder, f"{best_model_filename}.pkl")
best_rf_final = joblib.load(model_path)

In [ ]:
# Read embedding and check if the number of features is correct
src = rasterio.open(embedding_path)
print("Number of bands in raster:", src.count)
print("Number of features in model:", best_rf_final.n_features_in_)

In [ ]:
# Process raster in smaller blocks to save memory
block_shape = (512, 512)  # smaller block to be safe
predicted_path = os.path.join(output_folder, f"{prediction_filename}.tif")

# Open input raster
with rasterio.open(embedding_path) as src:
    profile = src.profile
    n_bands = src.count
    n_rows, n_cols = src.height, src.width

    # Prepare output raster profile: single band, uint8
    profile.update(count=1, dtype=rasterio.uint8)

    # Loop over raster in blocks to avoid loading entire raster into memory
    with rasterio.open(predicted_path, 'w', **profile) as dst_pred:
        for i in range(0, n_rows, block_shape[0]):
            for j in range(0, n_cols, block_shape[1]):
                try:
                    # Memory check (if needed, import psutil)
                    # mem = psutil.virtual_memory()
                    # print(f"Processing block at row {i}, col {j} | Available RAM: {mem.available/1e6:.1f} MB")

                    # Define window (subset of raster)
                    window = rasterio.windows.Window(
                        j, i,
                        min(block_shape[1], n_cols-j),
                        min(block_shape[0], n_rows-i)
                    )

                    # Read the block (n_bands, height, width)
                    block = src.read(window=window)  # shape: (n_bands, h, w)
                    n_b, h, w = block.shape

                     # Flatten block for prediction (pixels x features)
                    X_block = block.reshape(n_b, -1).T
                    # Convert to float32 to save memory
                    X_block = X_block.astype(np.float32)

                    # Optional: avoid DataFrame if memory is tight
                    # X_block = pd.DataFrame(X_block, columns=X.columns)

                     # Predict classes for all pixels in block
                    y_block_pred = best_rf_final.predict(X_block)

                    # Reshape predictions back to raster block shape
                    y_block_pred_raster = y_block_pred.reshape(h, w)

                     # Write predicted block to output raster
                    dst_pred.write(y_block_pred_raster, 1, window=window)

                    print(f"Block at row {i}, col {j} processed successfully")

                except Exception as e:
                    print(f"ERROR at block row {i}, col {j}: {e}")